# Document classification notebook

This notebook illustrate the lecture on document classification. See first lecture notebook or moodle for explanations on how to set up the proper environment. We will basically here make use of the same environment however augmented with pytorch.

## Dataset 

We will make use of the SST2 dataset from GLUE, a dataset dedicated to sentiment analysis. The task at hand is the classification of short utterances stating an opinion about a movue as expressing a positive (1) or a negative (0) opinion.

SST2 data was initially downloaded from https://dl.fbaipublicfiles.com/glue/data/SST-2.zip. Unfortunately doesn't have a label on the test data and there are discrepancies between train and dev, the latter having punctuation marks that the former does not have. We'll thus downsize the train set and split it into train, validation and test subsets. 

More info on the data at https://nlp.stanford.edu/sentiment/index.html.

Note that data preparation with SST2 is minimal as utterances are short, already cleaned with no punctuations and weird signs. 

If you want to explore further text classification and data preparation, you can also play with the IMDb dataset that contains movie reviews that are much longer than single utterances. See https://ai.stanford.edu/~amaas/data/sentiment for details. 

## Methods

The idea of this notebook is to illustrate a number of methods for text classification. We will guide you through: 
- an average embedding approach with a neural network classifier
- a recurrent neural network approach (in a subsequent lecture)

But you can easily implement and compare with the following two approaches:
- Naive Bayes classifier (e.g., with sklearn.naive_bayes.MultinomialNB)
- tf-idf vector space model with a k-nearest neighbor classifier (see illustration notebook on moodle)

In [ ]:
import csv
import json
import gzip
import numpy as np
from tqdm import tqdm
import statistics
from collections import Counter

# from sklearn.neighbors import NearestNeighbors
# from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from matplotlib import pyplot as plt

## Loading and tokenizing the dataset

Let's first load the data, clean a bit and tokenize. Last cell provides basic statistics on the corpus.

Punctuation marks were already removed in SST2 and all data is uncased so tokenization remains minimal here. It basically consists in separating tokens based on spaces after getting rid of weird symbols we don't want to bother with.

Note that in real life, if data hasn't been pre-processed as it is now, you'd have to invoke a real tokenizer such as the ones we made us of during the first lecture (e.g., NLTK sent_tokenize and word_tokenize, one of spaCy's pipeline). 

In [ ]:
#
# Let's first load raw data from the CSV files. 
#

import csv

def load_tsv_data(fn: str) -> list[dict]:
    '''
    Load data from file
    '''
    
    with open(fn, 'r') as f:
        dat = [x for x in csv.DictReader(f, delimiter="\t")] 

    return dat    
    
data = load_tsv_data('./data.NOSAVE/sst2-train.tsv')

for i in range(10):
    print('sample {} -- class={}, string=/{}/'.format(i, data[i]['label'], data[i]['sentence']))


In [ ]:
#
# Basic tokenization and cleansing of the utterances to classify. 
# 

def clean_and_tokenize_utterance(x: dict) -> dict:
    '''
    Tokenize utterance with basic rules and convert labels to int. Input an entry of the dataset as
    a dict() and returns the dictionary augmented with the list of tokens (as strings). Also converts
    the label to an integer.
    '''
    unwanted =  ("``", "''", "'", "`", "--", ",", ".")
    
    x['tokens'] = [token for token in x['sentence'].split() if token not in unwanted]
    x['label'] = int(x['label'])
    
    return x

# apply the utterance-level tokenizer to each data item
data = list(map(clean_and_tokenize_utterance, data))

for i in range(10):
    print('sample {} -- class={}, string=/{}/, tokens={}'.format(i, data[i]['label'], data[i]['sentence'], data[i]['tokens']))

# ========================================================
# TODO
# 
# Question: What do you think of the tokenization? Is it good, bad? What could cause problems later on 
# in the process of designing an utterance classifier?

In [ ]:
#
# Check some basic statistics on the data
#

nsamples = len(data)
nlabels = 2
nlabels0 = len([x for x in data if x['label'] == 0])
nlabels1 = len([x for x in data if x['label'] == 1])

ntokens = [len(x['tokens']) for x in data]

m = statistics.mean(ntokens)
m2 = statistics.median(ntokens)
sdev = statistics.stdev(ntokens)
    
print('{:35s} {}  {}/{}'.format('Number of samples in dataset', nsamples, nlabels0, nlabels1))
print('{:35s} min={}  max={}  mean={:.1f}  median={}  sdev={:.1f}'.format('Number of tokens per utterance', min(ntokens), max(ntokens), m, m2, sdev))


## Building the vocabulary

As we will design models that define statistics on tokens, we first have to build the vocabulary that we will be able to represent. A common choice when not using subword tokenization is to consider the most frequent tokens as the vocabulary.

We'll limit ourselves to simple terms (as opposed to complex terms such as 'can opener' or 'neural network') and simply select the most frequent terms in the corpus.

Note that many toolkits for NLP provide a sort of equivalent function, e.g., 
- gensim.corpora.dictionary.Dictionary -- https://radimrehurek.com/gensim/corpora/dictionary.html#
- tf.keras.preprocessing.text.Tokenizer -- https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
- torchtext.vocab -- https://pytorch.org/text/stable/vocab.html

But for pedagogical purposes, we'll do it once all by ourselves ;).


In [ ]:
#
# Check all tokens that appear in the dataset and count the number of times they appear.
#

counter = Counter()
   
for item in data:
    #
    # If we were to implement filters, e.g., on the POS tags, that's the place
    # where it could/should be done. We'd for instance create a temporary list
    # of relevant tokens before updating counter instead of taking all the tokens
    # as is.
    #
    counter.update(item['tokens'])

counter = dict(sorted(counter.items(), key=lambda x: x[1], reverse = True))

#
# Pretty print a number of things
#
print('total number of tokens in dataset =', len(counter))
print('most frequent tokens:')
for x in list(counter.keys())[:20]:
    print(f"   {x:18}  {counter[x]}")
print('\nleast frequent tokens:')
for x in list(counter.keys())[-20:]:
    print(f"   {x:18}  {counter[x]}")

# ========================================================
# TODO
# 
# Comment the most and least frequent tokens and think about how you could get
# a list of tokens of interest other than by selecting the most frequent ones.
#

In [ ]:
#
# Let's build the vocabulary from there, selecting the most frequent terms. Note that another common
# choice is to keep tokens that appear at least a certain number of times in the dataset.
#
# We will include two special tokens in the vocabulary:
#.  - PAD: the id of the padding token required for sequence models
#   - UNK: the id to assign to tokens that are not in the vocabulary
#
# The vocabulary will be dictionary mapping string to ids.
#

nterms = 4000 # number of terms to keep in the vocabulary

vocab = {'PAD': 0, 'UNK': 1} # initialize with the two special tokens before updating with actual regular tokens
offset = len(vocab)
vocab.update({x: i+offset for i,x in enumerate(list(counter.keys())[:nterms])})

print(len(vocab))
print(list(vocab.keys())[:20])

# for pretty printing later on, let's build a reverse mapping id2str from token IDs to the corresponding string
id2str = list(vocab.keys())

## Encoding and splitting data to create pytorch datasets and dataloaders

Last step before we can start building classifier, we have to split the dataset into train/validation/test and encode input sequences into list of integers rather than list of strings. 

We also have to convert each dataset into a pytorch Dataset that will be batched automatically through the Dataloader. 

### Encoding and splitting

In [ ]:
#
# Encoding the data simply means converting tokens from string to integer ids that are more suited for
# modeling and embedding. We will simply add a field 'ids' to each data item holding the list of token ids
#

def encode_utterance(x: dict) -> dict:
    '''
    Encode utterance according to the mapping provided by v. Input an entry of the dataset as
    a dict() and returns the dictionary augmented with the list of tokens (as strings). For 
    practical reasons, we're also adding a ids_no_unk field where unknown token ids are discarded.
    '''
    global vocab
    
    unk_id = vocab['UNK']
    
    x['ids'] = [vocab.get(token, unk_id) for token in x['tokens']] 
    x['ids_no_unk'] = [x for x in x['ids'] if x != unk_id]
    
    return x

data = list(map(encode_utterance, data))

print(data[0])


In [ ]:
#
# Let's finally split the data into three datasets
#

fold = dict()

fold['train'], buf = train_test_split(data, test_size=0.3, random_state=42)
fold['valid'], fold['test'] = train_test_split(buf, test_size=0.5, random_state=42)

for s in ('train', 'valid', 'test'):
    buf = fold[s]
    n0 = len([x for x in buf if x['label'] == 0])
    n1 = len([x for x in buf if x['label'] == 1])

    print('{:6s} = {}  {}/{}'.format(s, len(buf), n0, n1))

print(fold['train'][0])

In [ ]:
#
# Save dataset folds for later reuse
#

with gzip.open('data.NOSAVE/sst2-tokenized-folds.json.gz', 'wb') as f:
    json.dump(fold, f)
    
#
# reload
#
# with gzip.open('data.NOSAVE/sst2-tokenized-folds.json.gz', 'rb') as f:
#     fold = json.load(f)
#

### Turn datasets into a pytorch Dataset

Batching requires that all sequences have the same length, at least in a batch. 

In practice, this is ensured with a collator function that 0-pads the sequences on a per batch basis. To make it easier and more explicit, we will force all our sequences in the dataset to have the same length right from the start and don't bother with a collator. But we have to keep in mind this is suboptimal.

In [ ]:
import torch

from torch.utils.data import Dataset, DataLoader

In [ ]:
#
# Define the dataset class to hold the labels and the (padded) input ids referred to as 'encodings'.
#
# I chose to tokenize, encode and pad outside the dataset but this could all be embedded
# in the class constructor. See, e.g.,
# as in https://colab.research.google.com/github/DhavalTaunk08/NLP_scripts/blob/master/sentiment_analysis_using_roberta.ipynb#scrollTo=3vWRDemOGxJD
#
class MyDataset(Dataset):
    def __init__(self, encodings, labels):
        
        assert(len(encodings) == len(labels))
        
        self.nsamples = len(labels)
        
        # print(f'Initializing dataset with {self.nsamples} entries')
        
        self.encodings = encodings # list[list[int]]: contains the padded list of token ids for each sample
        self.labels = labels # list[int]: contains the label for each sample
        self.nlabels = len(set(labels)) # int: number of labels in the dataset

    def __getitem__(self, idx):
        '''
        Returns a dictionary containing the label and padded token ids for a sample
        '''
        
        # print(f'accessing dataset item at index {idx}')
        # print(torch.tensor(self.encodings[idx]), torch.tensor(self.labels[idx]))

        return {'ids': torch.tensor(self.encodings[idx]), 'label': torch.tensor(self.labels[idx])}

    def __len__(self):
        return len(self.labels)

#
# Define basic padding function
#
def pad_utterances(x: list, maxlen: int, pad_id: int = 0) -> list:
    '''
    Pad all input utterances up to maxlen, truncating if need be.
    
    Returns a list of padded ids. 
    '''
    
    return [(ids + [pad_id] * (maxlen - len(ids)))[:maxlen] for ids in x]


#
# Function to convert the dataset as a list[dict] into a proper torch.Dataset object
# 
def to_dataset(_data: list[dict], key: str = 'ids', maxlen: int = -1, pad_id: int = 0) -> MyDataset:
    '''
    Convert data as processed before into a proper pyTorch dataset with the specified tokenizer. 
    If maxlen <= 0, then we take the maximum length within the sequence.
    '''

    if maxlen <= 0:
        maxlen = max([len(x[key]) for x in _data])
        print('maxlen set to', maxlen)
        
    ids = [x[key] for x in _data]
    labels = [x['label'] for x in _data]
    encodings = pad_utterances(ids, maxlen)
    
    return MyDataset(encodings, labels)


In [ ]:
ds = dict()

ds['train'] = to_dataset(fold['train'], key='ids_no_unk', maxlen=10)
ds['valid'] = to_dataset(fold['valid'], key='ids_no_unk', maxlen=10)
ds['test'] = to_dataset(fold['test'], key='ids_no_unk', maxlen=10)

print('Training set:  nsamples =', ds['train'].nsamples, ' nlabels =', ds['train'].nlabels)

for i in range(3):
    print(fold['train'][i])
    print('   >>', ds['train'][i])


## A basic (toy) text classification neural network

Let's start text classification machinery with the basic model that we saw in the lecture that embeds the tokens, takes the average embedding and run a basic feed-forward classifier on top of it.

This section covers several steps in (pytorch) model design
1. define the model, i.e., its architecture
2. organize the dataset into batches, achieved with a Dataloader object in pytorch
3. define the training setup, i.e., optimizer and loss function along with a few parameters (e.g., number of epochs, learning rate)
4. run the training loop
5. evaluate results on the test set

In pytorch, the training loop has to be written explicitly (contrary to the fit() method in scikit-learn or tensorflow). Do does the evaluation loop (contrary to evaluate()). To facilitate things, two generic functions are given, namely train_step() and eval_step(): see below for more comments. These two functions are defined here since it's the first model we are conceiving but they will be used as is for the subsequent models.

### Creating the model and batches

Let's first create the model, batch the data via a DataLoader (this is where we're happy that all encoded sequences are of the same length) and illustrate how we can pass on data for inference.

In [ ]:
#
# Define the model as a torch.nn.Module
#
# Note that the softmax is not necessary when training with CrossEntropyLoss() which will
# apply the softmax to the logis. It is however required if training with NLLLoss().

class AvgEmbeddingNN(torch.nn.Module):

    def __init__(self, vocsize, nclasses = 2, embed_dim = 200, dropout = None):
        super(AvgEmbeddingNN, self).__init__()

        self.nclasses = nclasses
        self.vocabulary_size = vocsize
        self.embed_dim = embed_dim
        
        self.embedding = torch.nn.Embedding(vocsize, embed_dim, padding_idx = 0)
        self.dropout = torch.nn.Dropout(dropout) if dropout != None else None
        self.linear = torch.nn.Linear(embed_dim, nclasses)
        self.softmax = torch.nn.Softmax(dim=1)

    def forward(self, **kwargs):
        x = self.embedding(kwargs['ids']) # embed input ids -- batch_size * maxlen * embed_dim
        if self.dropout != None: # dropout embeddings
            x = self.dropout(x)        
        x = torch.mean(x, dim=1) # average embeddings yielding an average tensor -- batch_size * embed_dim
        x = self.softmax(self.linear(x)) # project into posterior probabilities -- batch_size * nclasses

        return x
        
model1 = AvgEmbeddingNN(len(vocab), nclasses = ds['train'].nlabels, embed_dim = 200)


In [ ]:
#
# Create batched dataset with data loaders
#

batch_size = 32

loader = dict()
loader['train'] = DataLoader(ds['train'], batch_size=batch_size, shuffle=True) # set to False for debugging purposes
loader['valid'] = DataLoader(ds['valid'], batch_size=batch_size)
loader['test'] = DataLoader(ds['test'], batch_size=batch_size)

print('Number of samples:', len(ds['train']))
print(f'Number of training batches:', len(loader['train']))

In [ ]:
# this cell is for illustration purposes only, showing how batches cab be passed and processed to
# the model's forward method at iunference time.

batch = next(iter(loader['train']))
print(batch['ids'].shape)
print(batch)

# Option 1: passing the ids tensor directly as a named argument
output = model1(ids=batch['ids'])
print(output.shape)
print(output)

# Option 2: passing an arbitrary number of named arguments, of which only ids=ids will be used
output = model1(**batch)

### Training the model

In pytorch, we have to explicitly write the training iterations. The code will vary little from one model to the other but still, we have to write the training loop explicitly. To simplify things, you are provided with two key functions defined in the following cells. The actual training and evaluation of the model follows.

#### Defining the basic building blocks of the training loop

Two key functions are provided here:

- train_step(): run one epoch given a model, a data loader, a loss function and an optimizer function; the function that should work for all models we'll be playing with

- eval_step(): evaluate the model's accuracy given a model, a data loader

We finally instantiate in the last cell of this section the various functions that we'll be needing for the training such as the optimizer, the loss function, etc.

In [ ]:
def train_step(_model, _loader, _loss, _optim, device="cpu", report=0):
    '''
    Generic training step.

    Assumes loader returns batches where the labels are accessed with the 'label' keyword.
    All other keywords are passed as **kwargs to the model.
    
    If report is set to a number, reports stats on training every 'report' batches.

    :return: total_loss accumulated throughout the epoch
    '''

    _model.train(True)
    total_loss = 0.
    running_loss = 0.

    for i, batch in enumerate(_loader):
        _optim.zero_grad()

        labels = batch['label'].to(device)

        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}
        outputs = _model(**inputs)

        loss = _loss(outputs, labels)
        total_loss += loss.item()
        running_loss += loss.item()

        loss.backward()
        _optim.step()

        if report != 0 and i % report == report - 1:
            print('  batch {} avg. loss per batch={:.4f}'.format(i + 1, running_loss / report))
            running_loss = 0.

    _model.train(False)

    return total_loss

In [ ]:
def eval_step(_model, _loader, device='cpu', loss_fn=None):
    '''
    Evaluate the model's performance on data within loader.
    
    :return: 
    total_loss accumulated throughout the batches
    accuracy
    '''
    
    # logger.debug('running eval_acc_recprec() on %s', device)

    _model.eval()  # disable training mode

    posteriors = torch.empty((0, _model.nclasses)).to(device)
    labels = torch.empty((0)).to(device)

    total_loss = 0.

    for batch in _loader:
        batch_labels = batch['label'].to(device)
        labels = torch.cat((labels, batch_labels), dim=0)

        inputs = {k: v.to(device) for k, v in batch.items() if k != 'label'}

        with torch.no_grad():
            outputs = _model(**inputs)

        posteriors = torch.cat((posteriors, outputs), dim=0)

        if loss_fn != None:
            loss = loss_fn(outputs, batch_labels)
            total_loss += loss.item()


    labels = labels.cpu()
    posteriors = posteriors.cpu()
    predictions = torch.argmax(posteriors, dim=-1)
    
    return total_loss, accuracy_score(labels, predictions)


In [ ]:
#
# Last but not least, we have to set the training parameters and instatiate all the objects 
# needed for training.
#

lr = 1e-4
nepochs = 10
report_freq = 200

# check what device we can work on
if torch.backends.mps.is_built(): # MPS GPU library for MacOS -- requires metal to be installed
    device = "mps"
    torch.mps.empty_cache()
elif torch.cuda.is_available(): # CUDA GPU acceleration available
    device = torch.device('cuda')
else:
    device = "cpu"
print(f'Running on {device} device')

optimizer = torch.optim.AdamW(model1.parameters(), lr=lr)
celoss = torch.nn.CrossEntropyLoss()

In [ ]:
#
# At last, here we go with the main training loop, iterating over epochs
#

model1.to(device)

for epoch in range(nepochs):
    print(f'epoch: {epoch}')
    
    total_loss = train_step(model1, loader['train'], celoss, optimizer, device=device, report=report_freq)
    _, trn_acc = eval_step(model1, loader['train'], device=device, loss_fn=None)
    
    val_loss, val_acc = eval_step(model1, loader['valid'], device=device, loss_fn=celoss)

    print('  **train** avg_loss={:.4f}    acuracy={:.2f}%'.format(total_loss / len(loader['train']), 100 * trn_acc))
    print('  **valid** avg_loss={:.4f}    acuracy={:.2f}%'.format(val_loss / len(loader['valid']), 100 * val_acc))


In [ ]:
_, tst_acc = eval_step(model1, loader['test'], device=device, loss_fn=None)

print('  **test** acuracy={:.2f}%'.format(100 * tst_acc))

In [ ]:
### TODO
#
# Visualize the word embeddings learned by the neural network with tSNE. Do we observe the 
# distributional semantic properties?
#
# Revisit the code above to work on lemmas rather than tokens, possibly limiting yourself to noun,
# verbs and adjectives which are the most relevant POS tags for polarity detection (adjectives alone 
# are already pretty good)
#

## Going further with the notebook

We will get back to this task after the lecture on recurrent neural networks and with large language models. 

In the meantime, you can try to improve by a better definition of the vocabulary. The choice of vocabulary is rather poor here and deserves significant improvement. For polarity detection, there's no need to work with "words" and lemmas are sufficient. Also for the approach based on average embeddings, function words (determiners, auxiliary, etc,) are not necessary and can be safely removed. You should adapt this first method so that the vocabulary is built from lemmatized input keeping only nouns, verbs and adjectives in the vocabulary. To do so, you need to run POS tagging and lemmatization (see first notebook on how to do that with NLTK for instance), adapt the vocabulary and hence the subsequent steps (preparation and encoding of the data).

You can experiment also with discarding / not discarding the UNK tokens and measure what impact it has.

Also, if you might want to compare with basic statistical methods based on symbolic word representations rather than embedding, for instance implementing a vector space model with tf-idf weights along with a k-nn classifier or a naive Bayes classifier.